In [1]:
import openai
import os
# Set keys
api_key="4b81012d55fb416c9e398f6149c3071e"
api_base =  "https://ey-sandbox.openai.azure.com/"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = api_key
openai.api_base = api_base
openai.api_type = 'azure'
openai.api_version = '2023-07-01-preview'
#openai.api_version = '2023-09-15-preview'

In [2]:
from langchain.schema import HumanMessage,SystemMessage
from langchain_openai import ChatOpenAI

def get_completion(prompt, engine="gpt-35-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=engine,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

  

In [3]:
get_completion("Translate this sentence from English to French: I love programming.")

"J'aime programmer."

In [4]:
def get_completion_m(prompt, engine="gpt-35-turbo", temperature=0):
    messages = [
        SystemMessage(
            content="You are a helpful assistant that translates English to French."
        ),
        HumanMessage(content="I love programming."),
    ]
    response = openai.ChatCompletion.create(
        engine=engine,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [8]:
from langchain.chains import ConversationChain
from langchain_community.chat_models.azure_openai import AzureChatOpenAI
chat=AzureChatOpenAI(openai_api_version="2023-07-01-preview", azure_deployment="gpt-35-turbo",)
                     
conversation = ConversationChain(llm=chat)
conversation.run("Translate this sentence from English to French: I love programming.")

'Je adore la programmation.'

In [10]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
chat([message])

C:\Users\PH735WK\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="J'aime programmer.")

In [11]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    chat([message])
    print(
        f"Total Cost (USD): ${format(cb.total_cost, '.6f')}"
    )  # without specifying the model version, flat-rate 0.002 USD per 1k input and output tokens is used

Total Cost (USD): $0.000040


In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [13]:
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: whats up?'}

In [14]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

In [21]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI
engine="gpt-35-turbo"
llm = AzureChatOpenAI(temperature=0,openai_api_key=api_key,model=engine)
memory = ConversationSummaryMemory(llm=chat)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context(
    {"input": "im working on better docs for chatbots"},
    {"output": "oh, that sounds like a lot of work"},
)
memory.save_context(
    {"input": "yes, but it's worth the effort"},
    {"output": "agreed, good docs are important!"},
)

In [22]:
memory.load_memory_variables({})

{'history': "The human greets the AI and they exchange pleasantries. The human mentions they are working on improving documentation for chatbots, and the AI acknowledges that it sounds like a lot of work. However, the human emphasizes that it's worth the effort, to which the AI agrees, stating that good documentation is important."}

In [23]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=chat, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [25]:
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)



# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=chat, prompt=prompt, verbose=True, memory=memory)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})

C:\Users\PH735WK\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you doing today?')],
 'text': 'Hello! How are you doing today?'}

In [26]:
conversation(
    {"question": "Translate this sentence from English to French: I love programming."}
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you doing today?
Human: Translate this sentence from English to French: I love programming.

> Finished chain.


{'question': 'Translate this sentence from English to French: I love programming.',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you doing today?'),
  HumanMessage(content='Translate this sentence from English to French: I love programming.'),
  AIMessage(content='"J\'aime programmer."')],
 'text': '"J\'aime programmer."'}

In [27]:
conversation({"question": "Now translate the sentence to German."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you doing today?
Human: Translate this sentence from English to French: I love programming.
AI: "J'aime programmer."
Human: Now translate the sentence to German.

> Finished chain.


{'question': 'Now translate the sentence to German.',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you doing today?'),
  HumanMessage(content='Translate this sentence from English to French: I love programming.'),
  AIMessage(content='"J\'aime programmer."'),
  HumanMessage(content='Now translate the sentence to German.'),
  AIMessage(content='"Ich liebe programmieren."')],
 'text': '"Ich liebe programmieren."'}

In [28]:
conversation({"question": "Now translate the sentence to Punjabi."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you doing today?
Human: Translate this sentence from English to French: I love programming.
AI: "J'aime programmer."
Human: Now translate the sentence to German.
AI: "Ich liebe programmieren."
Human: Now translate the sentence to Punjabi.

> Finished chain.


{'question': 'Now translate the sentence to Punjabi.',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you doing today?'),
  HumanMessage(content='Translate this sentence from English to French: I love programming.'),
  AIMessage(content='"J\'aime programmer."'),
  HumanMessage(content='Now translate the sentence to German.'),
  AIMessage(content='"Ich liebe programmieren."'),
  HumanMessage(content='Now translate the sentence to Punjabi.'),
  AIMessage(content='ਮੈਂ ਪ੍ਰੋਗਰਾਮਿੰਗ ਨੂੰ ਪਿਆਰ ਕਰਦਾ ਹਾਂ।')],
 'text': 'ਮੈਂ ਪ੍ਰੋਗਰਾਮਿੰਗ ਨੂੰ ਪਿਆਰ ਕਰਦਾ ਹਾਂ।'}

In [29]:
pip install --upgrade --quiet  tiktoken chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data=loader.load()

In [48]:
pip install unstructured unstructured[pdf] unstructured[all-docs] docx2txt

  Using cached docx2txt-0.8-py3-none-any.whl
     ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
      --------------------------------------- 0.3/19.2 MB 9.9 MB/s eta 0:00:02
     -- ------------------------------------- 1.2/19.2 MB 15.0 MB/s eta 0:00:02
     --- ------------------------------------ 1.8/19.2 MB 16.3 MB/s eta 0:00:02
     --- ------------------------------------ 1.8/19.2 MB 16.3 MB/s eta 0:00:02
     ----- ---------------------------------- 2.5/19.2 MB 12.1 MB/s eta 0:00:02
     ------- -------------------------------- 3.4/19.2 MB 14.4 MB/s eta 0:00:02
     ------- -------------------------------- 3.4/19.2 MB 14.4 MB/s eta 0:00:02
     -------- ------------------------------- 4.2/19.2 MB 12.8 MB/s eta 0:00:02
     ----------- ---------------------------- 5.4/19.2 MB 14.3 MB/s eta 0:00:01
     ------------ --------------------------- 6.0/19.2 MB 14.2 MB/s eta 0:00:01
     ------------- -------------------------- 6.7/19.2 MB 14.2 MB/s eta 0:00:01
    

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chainlit 0.7.700 requires dataclasses_json<0.6.0,>=0.5.7, but you have dataclasses-json 0.6.3 which is incompatible.
langchain-openai 0.0.3 requires openai<2.0.0,>=1.6.1, but you have openai 0.28.0 which is incompatible.
ydata-profiling 4.6.2 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.2 which is incompatible.
ydata-profiling 4.6.2 requires visions[type_image_path]==0.7.5, but you have visions 0.7.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/283.9 kB ? eta -:--:--
   -------------------------------------- - 276.5/283.9 kB ? eta -:--:--
   -------------------------------------- - 276.5/283.9 kB ? eta -:--:--
   ---------------------------------------- 283.9/283.9 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/239.6 kB ? eta -:--:--
   --------------------------------------  235.5/239.6 kB 15.0 MB/s eta 0:00:01
   --------------------------------------  235.5/239.6 kB 15.0 MB/s eta 0:00:01
   --------------------------------------  235.5/239.6 kB 15.0 MB/s eta 0:00:01
   --------------------------------------  235.5/239.6 kB 15.0 MB/s eta 0:00:01
   ---------------------------------------- 239.6/239.6 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/274.7 kB ? eta -:--:--
   -------------------------------------- - 266.2/274.7 kB ? eta -:--:--
   -------------------------------------- - 266.2/274.7 kB ? eta -:--:--
   ------------

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=0)
all_splits=text_splitter.split_documents(data)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
# from langchain_openai import AzureOpenAIEmbeddings
# from langchain_community.chat_models.azure_openai import AzureChatOpenAI
from langchain.embeddings import HuggingFaceInstructEmbeddings
from tqdm.autonotebook import trange
# embeddings = AzureOpenAIEmbeddings(
#     azure_deployment="gpt-35-turbo",
#     openai_api_version="2023-07-01-preview",
# )
embeddings = HuggingFaceInstructEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore=Chroma.from_documents(documents=all_splits,embedding=embeddings)

load INSTRUCTOR_Transformer
max_seq_length  512


In [50]:
pip install -U sentence-transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
pip install instructorembedding

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: instructorembedding in c:\users\ph735wk\myenv\lib\site-packages (1.0.1)




[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)

In [57]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
chat=AzureChatOpenAI(openai_api_version="2023-07-01-preview", azure_deployment="gpt-35-turbo",)
#llm = ChatOpenAI()
retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(chat, retriever=retriever, memory=memory)